#README.txt

Drag and drop a ZIPPED version of the folder `/data` from GitHub repo into Colab.

< - - - - -

In [1]:
# then run this and refresh directory...

# unzip datasets
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/job-postings/
   creating: data/resumes/
  inflating: data/README.txt         
  inflating: __MACOSX/data/._README.txt  
  inflating: data/job-postings/Data_Job_TX.csv  
  inflating: data/job-postings/Data_Job_WA.csv  
  inflating: data/job-postings/Data_Job_NY.csv  
  inflating: data/job-postings/Data_Job_SF.csv  
  inflating: data/resumes/kaggleResumes.csv  
  inflating: __MACOSX/data/resumes/._kaggleResumes.csv  




---


# API

In [2]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import os
import config
# API Docs found here: https://developer.usajobs.gov/Tutorials/Search-Jobs

In [4]:
config.US_JOBS_API_KEY = "xVc0TZiLfhcr17ci7Ngk6bLAetdRVFgntm2pZgWNtww="
config.EMAIL_ADDRESS = "gjacobthomas@gmail.com"

Note: I'm assuming this API has some limiter on it so we don't want to lose access. -tyler

In [5]:
host = 'data.usajobs.gov' 
# add these values in the config.py file
userAgent = config.EMAIL_ADDRESS
authKey = config.US_JOBS_API_KEY

base_url = "https://data.usajobs.gov/api/search"

parameters = {
    "JobCategoryCode": 2210,
    "Keyword": "Software Development",
    "LocationName": "Washington, DC"
}

headers = {
    "Host": host,          
    "User-Agent": userAgent,          
    "Authorization-Key": authKey  
}

resp = requests.request("GET", base_url,headers=headers, params=parameters)
result = resp.json()['SearchResult']['SearchResultItems']

# get Job Title 
print(result[1]['MatchedObjectDescriptor']['PositionTitle'])
# get Job Summary
print(result[1]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
# more parameters are found here: https://developer.usajobs.gov/API-Reference/GET-api-Search

Supervisory IT Program Manager (APPSW)/Assistant Director, Development Services
This position is located in Criminal Division's Office of Administration, Information Technology Management (ITM) unit and serves as the Assistant Director, Development Services. The Assistant Director has responsibility for the management of internal and external software services, including custom application development, intranet support, SharePoint, software service providers, and all related cyber security functions.




---


# KeyBERT Extraction Function

In [6]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.2 MB/s 
     |████████████████████████████████| 237 kB 3.8 MB/s 
     |████████████████████████████████| 51 kB 2.7 MB/s 
     |████████████████████████████████| 5.5 MB 31.4 MB/s 
     |████████████████████████████████| 1.3 MB 43.1 MB/s 
     |████████████████████████████████| 163 kB 46.3 MB/s 
     |████████████████████████████████| 7.6 MB 41.5 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23799 sha256=49cb10be64ab1f233c43d50ed06dda8d94e53ffe87bec757cf97e2096ab09d64
  Stored in directory: /root/.cache/pip/wheels/85/0d/12/77d219f3ebbb22dc22234b4d665886c0eace86a26eca0aa72b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2f8ac9fdbd9a9b1e7886bff7bef68d34d49a27223768f19f0d0b8a6f38aea8d2
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c

In [7]:
# Imports
from keybert import KeyBERT # pip install keybert (give it a minute...)

'''
/*---------------------------------------------------------------------
 |  Method: extractKeywordsBERT
 |
 |  Purpose: Uses the KeyBert Keyword Extraction Tool to extract
 |           and return keywords from a given corpus. 
 |      
 |  Author: Tyler Parks
 |  Created On: 10/30/22
 |
 |  Parameters:
 |      normalized_corpus -- A single string containing all text of the
 |                           normalized corpus.
 |
 |  Returns: 
 |      keywords -- List of collected keywords
 |      scores -- List of those keyword's scores
 |
 |  References: https://maartengr.github.io/KeyBERT/#usage
 |
 *-------------------------------------------------------------------*/
''' 
def extractKeywordsBERT(normalized_corpus):   
    print('---KeyBert Extraction---')
    print('------------------------\n')

    # init. language model 
    language_model = KeyBERT(model = 'all-mpnet-base-v2')

    # extract those keywords!
    data = language_model.extract_keywords( normalized_corpus, 
                                            keyphrase_ngram_range=(1, 3), 
                                            stop_words='english',
                                            use_maxsum=False, 
                                            use_mmr=True,
                                            diversity=0.7,
                                            nr_candidates=20, 
                                            top_n=15
                                        )

    # zip the lists
    zipped = list(map(list, zip(*data)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    print()

    return keywords, scores



---


# CSO-Classifier Extraction Function

In [8]:
!python -m spacy download en_core_web_sm
!pip install cso-classifier

2022-11-07 01:17:04.504905: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 2.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 61 kB 6.6 MB/s 
     |████████████████████████████████| 24.2 MB 75.1 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 1.5 MB 45.0 MB/s 
     |████████████████████████████████| 3.2 MB 60.0 MB/s 
     |████████████████████████████████| 50 kB 1.7 MB/s 
     |████████████████████████████████| 12.8 MB 45.3 MB/s 
     |█████████████████

**Don't restart runtime if the terminal says so! Keep going.**

In [9]:
import nltk
nltk.download('stopwords')
# update spacy
!pip install -U spacy
import spacy
from cso_classifier import CSOClassifier      # import classifier tool

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 1.8 MB/s 
     |████████████████████████████████| 806 kB 10.5 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.5
    Uninstalling spacy-3.0.5:
      Successfully uninstalled spacy-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cso-classifier 3.0 requires spacy==3.0.5, but you have spacy 3.4.2 which is incompatible.


In [10]:
# update the most recent model
CSOClassifier.update() 

# define the model object
CSO_Extractor = CSOClassifier(modules = "both", enhancement = "first", explanation = False)


# ======================================================
#     ONTOLOGY
# ======================================================
Updating the ontology file
[██████████████████████████████████████████████████] 7M/7M
[*] Done!
Extracting and converting ontology.
Creating ontology pickle file from a copy of the CSO Ontology found in /usr/local/lib/python3.7/dist-packages/cso_classifier/assets/cso.csv
Creating graph representation of the ontology.
Saving the graph representation of the ontology (in a pickle object) in /usr/local/lib/python3.7/dist-packages/cso_classifier/assets/cso_graph.p

# ======================================================
#     MODELS: CACHED & WORD2VEC
# ======================================================
Couldn't delete file cached model: not found
Couldn't delete file word2vec model: not found
Updating the models: cached and word2vec
[██████████████████████████████████████████████████] 63M/63M
[*] Done!
[██████████████████████████████████████████████████] 34

In [11]:
def extractKeywordsCSO(normalized_corpus):

  # run the extraction
  result = CSO_Extractor.run(normalized_corpus)

  print('-----CSO Extraction-----')
  print('------------------------\n')
  
  for keyword in result['union']:
    print(keyword)

  return result['union']

# **YAKE Extractor**

In [12]:
!pip3 install yake
import yake 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.4 MB/s 
     |████████████████████████████████| 132 kB 10.3 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73984 sha256=abb132bb78c9fcd1d3917df735b492fcf5455f8918e35bea80e16f07f1e57ab9
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [13]:
def extractKeywordsYAKE(normalized_corpus):
    print('---YAKE Extraction---')
    print('------------------------\n')

    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = kw_extractor.extract_keywords(normalized_corpus)

    zipped = list(map(list, zip(*keywords)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    
    return keywords, scores

# **Levenstein Distance Matching**

In [14]:
!pip install jellyfish # the jellyfish library is used to calculate the Levenstein distance of two strings.
!pip install gensim
!pip install jieba
import jellyfish
from functools import reduce
import gensim
import nltk
from nltk.tokenize import word_tokenize
import jieba
nltk.download('punkt')
import re
import numpy as np

class SkillsMatching:
  def __init__(self, all_job_postings_keywords, all_resumes_keywords):
      self.all_job_postings_keywords = all_job_postings_keywords
      self.all_resumes_keywords = all_resumes_keywords

  # output JSON with matching job postings for each resume
  def skill_match(self):
      results = {"matching_job": []}
      for resume_json in self.all_resumes_keywords:
          # need to change this to match all text classifiers
          resume_keywords = resume_json["keywordsBERT"]
          scores = self.sim_score(self.all_job_postings_keywords, resume_keywords)
          # grab the index of the largest score in the arr
          max_idx = np.argmax(scores)
          results['matching_job'].append(self.all_job_postings_keywords[max_idx])

      return results

  def split_and_join_arr(self, arr):
    new_arr = []
    for w in arr:
      word_arr = re.split('\W+', w.lower())
      new_arr = new_arr + word_arr
    # print(new_arr)
    return new_arr
  
  def sim_score(self, docs, keywords):
    keywords = self.split_and_join_arr(keywords)
    gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in docs]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity('/usr/workdir',tf_idf[corpus],
                                      num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(sims[query_doc_tf_idf])
    return sims[query_doc_tf_idf]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.




---


#Driver Code

In [15]:
# Imports
import pandas as pd     # pip install pandas. usage: loading data from csv files into dataframes
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("all")

In [16]:
### Helper Functions

# Function to retrieve text data.
# (either 1 or more job postings or resumes)?
def getFileData(filename, dir):
    return pd.read_csv('data/' + dir + '/' + filename)

# Function to normalize text data. 
# (some skill extraction tools will normalize text for us; however, if not, this function is here)
# includes removing stopwords, punctuation, dates, links, etc...
def normalizeCorpus(corpus):
    return corpus
    # need to fix this 
    # nltk_tokenList = word_tokenize(corpus)
    # lemmatizer = WordNetLemmatizer()
    # nltk_lemmaList = []
    # for word in nltk_tokenList:
    #     nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # normalized_corpus = []  
    # nltk_stop_words = set(stopwords.words("english"))
    # for w in nltk_lemmaList:  
    #     if w not in nltk_stop_words:  
    #         normalized_corpus.append(w)

    # punctuation = ";:.,?!"
    # for word in normalized_corpus:
    #     if word in punctuation:
    #         normalized_corpus.remove(word)

    #still need to add dates and links
    # return normalized_corpus

# Function to extract skill words from a given corpus.
# ideally, this function will output a set of skills extracted from the corpus
def extractSkills(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO              = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE = extractKeywordsYAKE(corpus)
    # extraction method n...
    # keep going!
    return {"keywordsBERT": keywordsBERT,
            "keywordsCSO": keywordsCSO,
            "keywordsYAKE": keywordsYAKE}

### Driver Code
if __name__ == '__main__':

    # fetch the data
    job_posting_dataframe = getFileData('Data_Job_TX.csv'  , 'job-postings')
    resume_dataframe      = getFileData('kaggleResumes.csv', 'resumes'     )
    #---------------

    '''
    # print the dataframes
    print('DataFrame of Job Postings:')
    print(job_posting_dataframe)    
    print()

    print('DataFrame of Resumes:')
    print(resume_dataframe)
    print()
    #----------------------
    '''

    # fetch the job descriptions and resumes by themselves
    jpCorpus = list(job_posting_dataframe['Job_Desc'])
    rCorpus  = list(resume_dataframe['Resume'])
    #----------------------

    # Number of both job posting and resume samples to view
    NUM_SAMPLES = 10

    # for each JOB POSTING from the corpus
    i = 0
    all_job_postings = []
    for posting in jpCorpus:
        print('Job Posting #', i+1)
        print()

        text = normalizeCorpus(posting)
        job_posting_json = extractSkills(text)
        all_job_postings.append(text)
        # print lines, we are done with this posting
        print('------------------------\n')

        # break, after X postings
        i += 1
        if i > NUM_SAMPLES:
            break
    #---------------------- 

    # for each RESUME from the corpus
    i = 0
    all_resumes_keywords = []
    for resume in rCorpus:
        print('Resume #', i+1)
        print()

        text = normalizeCorpus(resume)
        resume_json = extractSkills(text)
        all_resumes_keywords.append(resume_json)
        # print lines, we are done with this resume
        print('------------------------\n')

        # break, after X resumes
        i += 1
        if i > NUM_SAMPLES:
            break
    #---------------------- 

    
# keep going!

# end of driver code
#---------------

Job Posting # 1

---KeyBert Extraction---
------------------------



Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

-Skill-                                  -Score-
senior data scientist                    0.6052
products alarm monitoring                0.3292
summary brinks home                      0.281
forest regression designing              0.2549
join team trusted                        0.2381
azure aws google                         0.2262
acquisition customer retention           0.2061
sentiment analysis gradient              0.177
efforts requirements                     0.1714
clearly communicate model                0.151
true                                     0.1165
libraries thorough understanding         0.114
excellent                                0.1049
america                                  0.0347
action true responsive                   0.0209

Computer Science Ontology loaded.
Model loaded.


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


-----CSO Extraction-----
------------------------

optimization
customer retention
computer science
business intelligence
smart homes
mathematics
visualization
engineers
boosting
random forests
text mining
correlation analysis
machine learning
communication
sentiment analysis
sales
engineering
cloud infrastructures
applied mathematics
---YAKE Extraction---
------------------------

-Skill-                                  -Score-
North America                            0.0039669070818620965
providing cutting-edge products          0.005520810261696447
alarm monitoring services                0.005897400491803465
residential security industries          0.009126705858962662
Senior Data Scientist                    0.010922433704422788
Brinks Home Security                     0.01237827812123832
smart home technology                    0.021945694982601
providing cutting-edge                   0.02983093214339939
proven leader                            0.03182037980964527
technology an

In [18]:
# Resume and Job Posting Matching
matching = SkillsMatching(all_job_postings, all_resumes_keywords)
match_job_postings = matching.skill_match()
# print(match_job_postings)
for i, jp in enumerate(match_job_postings['matching_job']):
  i += 1
  print('------------------------\n')
  print("Matching Job Posting for Resume %s" % i)
  print(jp)

------------------------

Matching Job Posting for Resume 1
The Data Analyst II is responsible for data entry and maintenance for our integrated enterprise system (SAP). As part of the Data Integrity team, this position is responsible for the accurate entry of data into SAP. The Data Integrity team is responsible for the efficient and effective management of part master data. This will require an individual with exceptional attention to detail, intermediate to advanced computer skills, in-depth knowledge of our business, products and processes.

WORK PERFORMED

To perform this job successfully, an individual must be able to perform each essential duty satisfactorily. The requirements listed below are representative of the knowledge, skill, and/or ability required.

Essential Functions of the Job:

Product Data – Responsible for managing product information levels to established standards

Maintain data to meet company standards


Data includes images, technical specifications and other